In [41]:
import os
import pandas as pd
import numpy as np
import json
from datasets import Dataset
from tqdm import tqdm
import ast

import evaluate as hf_evaluate
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

In [43]:
GENERATION_FILE_PATH = "/Users/connie/Desktop/Fall 2025/LLM/Assignment2/data/evaluation/all_experiment_generations.csv"
RESULTS_DIR = os.path.join("..", "results")
os.makedirs(RESULTS_DIR, exist_ok=True)

In [45]:
print("Reading generated data from:", GENERATION_FILE_PATH)
if not os.path.exists(GENERATION_FILE_PATH):
    raise FileNotFoundError("ERROR: Generation file not found. Please run naive_rag.py first.")

qa_df = pd.read_csv(GENERATION_FILE_PATH)
qa_df["answer"] = qa_df["answer"].astype(str) 

Reading generated data from: /Users/connie/Desktop/Fall 2025/LLM/Assignment2/data/evaluation/all_experiment_generations.csv


In [47]:
qa_df.columns

Index(['question', 'answer', 'q_len', 'q_bin', 'gen_naive_k1_all-MiniLM-L6-v2',
       'gen_cot_k1_all-MiniLM-L6-v2', 'gen_persona_k1_all-MiniLM-L6-v2',
       'gen_naive_k3_all-MiniLM-L6-v2', 'gen_cot_k3_all-MiniLM-L6-v2',
       'gen_persona_k3_all-MiniLM-L6-v2', 'gen_naive_k5_all-MiniLM-L6-v2',
       'gen_cot_k5_all-MiniLM-L6-v2', 'gen_persona_k5_all-MiniLM-L6-v2',
       'gen_naive_k1_all-mpnet-base-v2', 'gen_cot_k1_all-mpnet-base-v2',
       'gen_persona_k1_all-mpnet-base-v2', 'gen_naive_k3_all-mpnet-base-v2',
       'gen_cot_k3_all-mpnet-base-v2', 'gen_persona_k3_all-mpnet-base-v2',
       'gen_naive_k5_all-mpnet-base-v2', 'gen_cot_k5_all-mpnet-base-v2',
       'gen_persona_k5_all-mpnet-base-v2', 'contexts'],
      dtype='object')

In [49]:
# Define embeddings model (used by RAGAS)
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

### Finding out the Basic QA Metrics (F1 score, EM score)

In [52]:
print("\n--- Running Basic QA Metrics (F1/EM) ---")

# Get columns to be evaluated (e.g., generated_naive_k1, generated_cot_k1)
generation_cols = [c for c in qa_df.columns if c.startswith('gen_')]
final_metrics_summary = []

for col in tqdm(generation_cols, desc="Calculating F1/EM"):
    
    # Extract strategy name (e.g., 'naive') and K value (e.g., 1) from column name
    parts = col.split('_')
    k_part = parts[-2] 
    retrieval_k = int(k_part.replace('k', ''))
    strategy_name = parts[1]
    embedding_dim = parts[-1] 
    
    metric_f1 = hf_evaluate.load("squad") 
    f1_scores, em_scores = [], []
    
    for pred_full, truth in zip(qa_df[col], qa_df["answer"]):
        
        # 1. Answer Extraction (Process CoT output format)
        pred = pred_full
        if strategy_name == "cot" and isinstance(pred_full, str) and "Final Answer:" in pred_full:
              # Extract text after the CoT marker
              pred = pred_full.split("Final Answer:")[1].strip()
        
        # 2. Calculate F1 and EM score
        result = metric_f1.compute(
            predictions=[{"prediction_text": str(pred), "id": "0"}],
            references=[{"answers": {"text": [str(truth)], "answer_start": [0]}, "id": "0"}]
        )
        f1_scores.append(result["f1"])
        em_scores.append(result["exact_match"])
        
    avg_f1, avg_em = np.mean(f1_scores), np.mean(em_scores)

    # Temporarily store F1/EM results
    final_metrics_summary.append({
        "Retrieval_K": retrieval_k,
        "Prompt_Strategy": strategy_name,
        "Embedding_Dim": embedding_dim,
        "Avg_F1": avg_f1,
        "Avg_EM": avg_em,
    })


--- Running Basic QA Metrics (F1/EM) ---


Calculating F1/EM: 100%|████████████████████████| 18/18 [00:16<00:00,  1.06it/s]


In [53]:
# Convert results into DataFrame
results_df = pd.DataFrame(final_metrics_summary)

# Save to CSV
csv_path = os.path.join(RESULTS_DIR, "evaluation_f1_em.csv")
results_df.to_csv(csv_path, index=False)

# Pretty print top results (sorted by F1)
print("\n=== Evaluation Results (Top 10 by F1) ===")
print(results_df.sort_values(by="Avg_F1", ascending=False).head(10).to_string(index=False))

print(f"\n✅ Results saved to {csv_path}")


=== Evaluation Results (Top 10 by F1) ===
 Retrieval_K Prompt_Strategy     Embedding_Dim    Avg_F1    Avg_EM
           5         persona all-mpnet-base-v2 49.857943 40.000000
           5         persona  all-MiniLM-L6-v2 49.002727 39.166667
           3         persona  all-MiniLM-L6-v2 46.555274 37.500000
           3         persona all-mpnet-base-v2 44.691283 35.833333
           5           naive all-mpnet-base-v2 41.772417 32.500000
           5           naive  all-MiniLM-L6-v2 40.442060 30.833333
           5             cot  all-MiniLM-L6-v2 38.771223 25.000000
           5             cot all-mpnet-base-v2 38.412203 26.666667
           1         persona  all-MiniLM-L6-v2 38.208599 30.833333
           3             cot  all-MiniLM-L6-v2 37.215111 25.833333

✅ Results saved to ../results/evaluation_f1_em.csv


### Advanced Evaluation using RAGAs

In [ ]:
from dotenv import load_dotenv
load_dotenv()

llm_openai = ChatOpenAI(
    model="gpt-4o-mini",
    max_retries=5, # Increase retries to handle rate limits gracefully
)
hf_llm = llm_openai


In [10]:
# Parse contexts into clean lists of passage strings
def parse_context(ctx_str):
    try:
        ctx_obj = ast.literal_eval(ctx_str)  # safely parse string
        if isinstance(ctx_obj, dict) and "passage" in ctx_obj:
            return [ctx_obj["passage"]]
        elif isinstance(ctx_obj, list):
            return [c["passage"] for c in ctx_obj if isinstance(c, dict) and "passage" in c]
    except Exception:
        return []
    return []

qa_df["contexts_clean"] = qa_df["contexts"].apply(parse_context)

In [37]:
# ----------------------------------------------------
# STEP 1: Stratified sampling (done ONCE for all columns)
# ----------------------------------------------------
qa_df["q_len"] = qa_df["question"].apply(lambda x: len(str(x).split()))

# Define bins (short, medium, long) using percentiles
bins = np.percentile(qa_df["q_len"], [33, 66])
qa_df["q_bin"] = pd.cut(
    qa_df["q_len"],
    bins=[-1, bins[0], bins[1], float("inf")],
    labels=["short", "medium", "long"]
)

# Sample 40 from each bin (same random_state → deterministic)
subset_df = qa_df.groupby("q_bin", group_keys=False).apply(
    lambda x: x.sample(min(40, len(x)), random_state=42)
).reset_index(drop=True)

print(f"\n--- Using {len(subset_df)} stratified queries for ALL columns ---")
print(subset_df["q_bin"].value_counts())

# ----------------------------------------------------
# STEP 2: Run RAGAS per column, but on same 120 queries
# ----------------------------------------------------
chunk_size = 10  # row-level logging still works

for i, summary_data in enumerate(tqdm(final_metrics_summary, desc="Running RAGAS")):
    col = generation_cols[i]
    print(f"\nEvaluating column: {col}")

    metric_sums = {"faithfulness": 0, "answer_relevancy": 0,
                   "context_precision": 0, "context_recall": 0}
    total_rows = 0

    for start in range(0, len(subset_df), chunk_size):
        end = min(start + chunk_size, len(subset_df))
        print(f"{datetime.now().strftime('%H:%M:%S')} | Processing rows {start}–{end}...")

        for idx in range(start, end):
            row_data = Dataset.from_dict({
                "question": [str(subset_df["question"].iloc[idx])],
                "answer": [str(subset_df[col].iloc[idx])],
                "contexts": [subset_df["contexts_clean"].iloc[idx]],
                "ground_truths": [[str(subset_df["answer"].iloc[idx])]],
                "reference": [str(subset_df["answer"].iloc[idx])],
            })

            try:
                res = evaluate(
                    row_data,
                    metrics=[faithfulness, answer_relevancy,
                             context_precision, context_recall],
                    llm=hf_llm,
                    embeddings=hf_embeddings,
                    show_progress=False,
                    batch_size=1
                )

                for k in metric_sums.keys():
                    val = res[k]
                    if isinstance(val, list):  # ragas sometimes returns [score]
                        val = val[0]
                    metric_sums[k] += val
                total_rows += 1

                print(f"   Done row {idx+1}/{len(subset_df)} | {res}")

            except Exception as e:
                print(f"   Row {idx+1} failed: {e}")

    if total_rows > 0:
        for k in metric_sums.keys():
            metric_sums[k] /= total_rows
        summary_data.update({
            "Embedding_Dim": final_metrics_summary[i]['Embedding_Dim'],
            "Ragas_Faithfulness": metric_sums["faithfulness"],
            "Ragas_Answer_Relevancy": metric_sums["answer_relevancy"],
            "Ragas_Context_Precision": metric_sums["context_precision"],
            "Ragas_Context_Recall": metric_sums["context_recall"]
        })
        print(f"{datetime.now().strftime('%H:%M:%S')} | Completed {col} with averages: {metric_sums}")
    else:
        print(f"{datetime.now().strftime('%H:%M:%S')} | No valid rows processed for {col}")
        summary_data.update({
            "Ragas_Faithfulness": None,
            "Ragas_Answer_Relevancy": None,
            "Ragas_Context_Precision": None,
            "Ragas_Context_Recall": None
        })

/var/folders/k3/vcbwsjp93rl2cww52jf518fc0000gn/T/ipykernel_15058/1150768349.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  subset_df = qa_df.groupby("q_bin", group_keys=False).apply(
/var/folders/k3/vcbwsjp93rl2cww52jf518fc0000gn/T/ipykernel_15058/1150768349.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  subset_df = qa_df.groupby("q_bin", group_keys=False).apply(



--- Using 120 stratified queries for ALL columns ---
q_bin
short     40
medium    40
long      40
Name: count, dtype: int64


Running RAGAS:   0%|                                     | 0/18 [00:00<?, ?it/s]


Evaluating column: gen_naive_k1_all-MiniLM-L6-v2
00:45:13 | Processing rows 0–10...
   Done row 1/120 | {'faithfulness': 0.0000, 'answer_relevancy': 0.1109, 'context_precision': 0.0000, 'context_recall': 0.0000}
   Done row 2/120 | {'faithfulness': 0.0000, 'answer_relevancy': 0.0638, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 3/120 | {'faithfulness': 0.0000, 'answer_relevancy': 0.1015, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 4/120 | {'faithfulness': 0.0000, 'answer_relevancy': 0.0827, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 5/120 | {'faithfulness': 0.0000, 'answer_relevancy': 0.0778, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 6/120 | {'faithfulness': 1.0000, 'answer_relevancy': 0.0655, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 7/120 | {'faithfulness': 0.0000, 'answer_relevancy': 0.0310, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 8/120 |

Running RAGAS:   6%|█▏                    | 1/18 [1:24:29<23:56:20, 5069.45s/it]

...

   Done row 120/120 | {'faithfulness': 0.0000, 'answer_relevancy': 0.0631, 'context_precision': 1.0000, 'context_recall': 1.0000}
23:42:43 | Completed gen_naive_k5_all-mpnet-base-v2 with averages: {'faithfulness': 0.0375, 'answer_relevancy': 0.056595967357545726, 'context_precision': 0.716666666594999, 'context_recall': 0.6166666666666667}

Evaluating column: gen_cot_k5_all-mpnet-base-v2
23:42:43 | Processing rows 0–10...
   Done row 1/120 | {'faithfulness': 0.0000, 'answer_relevancy': 0.7123, 'context_precision': 0.0000, 'context_recall': 0.0000}
   Done row 2/120 | {'faithfulness': 1.0000, 'answer_relevancy': 0.8451, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 3/120 | {'faithfulness': 0.3333, 'answer_relevancy': 0.0000, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 4/120 | {'faithfulness': 1.0000, 'answer_relevancy': 0.3133, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 5/120 | {'faithfulness': 1.0000, 'answer_relevancy

Running RAGAS:  94%|███████████████████▊ | 17/18 [24:25:58<1:26:40, 5200.56s/it]

   Done row 120/120 | {'faithfulness': 0.0000, 'answer_relevancy': 0.7122, 'context_precision': 1.0000, 'context_recall': 1.0000}
01:11:12 | Completed gen_cot_k5_all-mpnet-base-v2 with averages: {'faithfulness': 0.5104166666666666, 'answer_relevancy': 0.5172491761386305, 'context_precision': 0.7333333332599989, 'context_recall': 0.6083333333333333}

Evaluating column: gen_persona_k5_all-mpnet-base-v2
01:11:12 | Processing rows 0–10...
   Done row 1/120 | {'faithfulness': 0.0000, 'answer_relevancy': 0.0000, 'context_precision': 0.0000, 'context_recall': 0.0000}
   Done row 2/120 | {'faithfulness': 1.0000, 'answer_relevancy': 0.8618, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 3/120 | {'faithfulness': 0.2500, 'answer_relevancy': 0.7570, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 4/120 | {'faithfulness': 1.0000, 'answer_relevancy': 0.2225, 'context_precision': 1.0000, 'context_recall': 1.0000}
   Done row 5/120 | {'faithfulness': 1.0000, 'ans

Running RAGAS: 100%|███████████████████████| 18/18 [25:52:51<00:00, 5176.17s/it]

   Done row 120/120 | {'faithfulness': 1.0000, 'answer_relevancy': 0.0000, 'context_precision': 1.0000, 'context_recall': 1.0000}
02:38:04 | Completed gen_persona_k5_all-mpnet-base-v2 with averages: {'faithfulness': 0.6125, 'answer_relevancy': 0.38534648149643247, 'context_precision': 0.716666666594999, 'context_recall': 0.625}


### Final Results Storage

In [38]:
# Define results directory explicitly (one level up from src/)
RESULTS_DIR = os.path.join("..", "results")
os.makedirs(RESULTS_DIR, exist_ok=True)

# Save the sampled subset queries (for reproducibility and appendix)
subset_path = os.path.join(RESULTS_DIR, "subset_queries.csv")
subset_df.to_csv(subset_path, index=False)
print(f"\nSampled queries saved to {subset_path}")

# Convert evaluation results into DataFrame
results_df = pd.DataFrame(final_metrics_summary)

# Save results to CSV
csv_path = os.path.join(RESULTS_DIR, "comparison_analysis.csv")
results_df.to_csv(csv_path, index=False)

print("\nEvaluation Finalized.")
print(f"Metrics saved to {json_path} and {csv_path}")


Sampled queries saved to ../results/subset_queries.csv

Evaluation Finalized.
Metrics saved to ../results/naive_results.json and ../results/comparison_analysis.csv
